In [ ]:
#Projet Mogpl M1 Informatique
#Dice Battle

#Yuhan WANG & Yufei LIU

In [152]:
import numpy as np
import matplotlib.pyplot as plt
import math

#variable global N est le point cible et D est le nombre de dés qu'on peut lancer maximument chaque fois
#LP = ligne perdu signifie que si joueur1 a moins de cette ligne et joueur 2 a N-1 point alors joueur1 a perdu
N = 100
D = 10
LP = N - 6*D

#Q1
def initialise_tab(d):
    """
    entrée : le nombre de dés
    sortie : Q_tab et P_tab initialisé
    """
    Q_tab=np.ones((D+1,101))*-1
    
    for a in range(0,101):
        Q_tab[0,a]=0
    for b in range(0,D+1):
        Q_tab[b,0]=0
    for i in range (2,7):
        #la propabilité d'obtenir 1 avec d dés
        Q_tab[1,i]=1/5
    
    P_tab=np.ones((D+1,101))*-2
    for c in range(0,D+1):
        P_tab[c,0]=0
        P_tab[c,1]=1-np.power(5/6,c)
    for d in range(0,101):
        P_tab[0,d]=0
        
        
    return Q_tab,P_tab  

Q_tab=initialise_tab(D)[0]
P_tab= initialise_tab(D)[1]

def Q(d,k):
    if d<=0:
        return 0
    if k<=0 | ((k>1) & (k<2*d)) | k>6*d :
        return 0
    else:
        if(Q_tab[d,k]!=-1):
            #print(d,k,Q_tab[d,k])
            return Q_tab[d,k]
        else:
            res = 0
            for j in range(2,7):
                #print(d-1,k-j)
                if(Q_tab[d-1,k-j]==-1):
                    Q_tab[d-1,k-j]=Q(d-1,k-j)
                res += Q(d-1,k-j)
            return res/5
        
        
def P(d,k):
    if(k==1):
        return P_tab[d,1]
    if P_tab[d,k]!=-2:
        return P_tab[d,k]
    else:
        P_tab[d,k]=(5/6)**d*Q(d,k)
        return P_tab[d,k]

In [153]:
#Q2

def calcul_EP(d):
    return 4*d*(5/6)**d + 1 - (5/6)**d

print(calcul_EP(5))
print(calcul_EP(6))
#On doit choisir d*=6 car EP(6) est supérieur de EP(5)




8.63567386831276
8.702653463648836


In [154]:
#白给的
np.set_printoptions(threshold=np.inf)

def EG(i,j):
    lim = N-1+6*D
    if (i>=N) | ((i==N-1) & (j<N)):
        return 1
    elif (i<LP & j==N-1) | j>=N:
        return -1
    else:
        tab = np.zeros(((lim-i+1),(lim-j+1)))
                
        for a in range(lim-i+1):
            for b in range(lim-j+1):
                if(a<6*D):
                    tab[a][b]=1
                elif(a==6*D):
                    if(b>=6*D):
                        tab[a][b]=1
                    else:
                        tab[a][b]=-1
                elif((6*D<a)&(b<6*D)):
                    tab[a][b]=-1
        #print("heelo",tab)
        for c in range(6*D+1,lim-i+1):
            for d in range(6*D,lim-j+1):
                if tab[c][d]==0:
                    cas_j1 = []
                    for d1 in range(1,D+1):
                        res1 = 0
                        for e1 in range(1,6*d1+1):
                            cas_j2 = []
                            for d2 in range(1,D+1):
                                res2 = 0
                                for e2 in range(1,6*d2+1):
                                    res2 += tab[c-e1][d-d2] * tab_P[d2-1][e2-1]
                                cas_j2.append(res2)
                            opt_j2 = np.argmin(np.array(cas_j2))
                            res1 += cas_j2[opt_j2] * tab_P[d1-1][e1-1]
                        cas_j1.append(res1)
                tab[c][d]=max(cas_j1)  
        #print("bye",tab.shape,tab)
        return tab[lim-i][lim-j]
        
              
def EG2(i,j):
    lim = N-1+6*D
    if (i>=N) | ((i==N-1) & (j<N)):
        return 1
    elif (i<LP & j==N-1) | j>=N:
        return -1
    else:
        tab = np.zeros(((lim-i+1),(lim-j+1)))
                
        for a in range(lim-i+1):
            for b in range(lim-j+1):
                if(a<6*D):
                    tab[a][b]=1
                elif(a==6*D):
                    if(b>=6*D):
                        tab[a][b]=1
                    else:
                        tab[a][b]=-1
                elif((6*D<a)&(b<6*D)):
                    tab[a][b]=-1
        #print("heelo",tab)
        for c in range(6*D+1,lim-i+1):
            for d in range(6*D,lim-j+1):
                if tab[c][d]==0:
                    ress=[]
                    for d1 in range(1,D+1):
                        cum=0
                        for e1 in range(1,6*d1+1):
                            print("i",i)
                            cum += tab_P[d-1][e1-1] * tab[d][c-e1]
                        ress.append(-cum)
                
                tab[c][d]=max(ress)  
        #print("bye",tab.shape,tab)
        return tab[lim-i][lim-j]

    



In [161]:
class strategie_optimal(object):
    def __init__(self, D, N):
        self.N = N
        self.D = D
        self.tab = np.ones((N,N))*100
         
    def EG(self,i,j):
        if(i>=self.N):
            return 1
        elif(j>=self.N):
            return -1
        elif self.tab[i][j]<100:
            return self.tab[i][j]
        
        ress=[]
        for d1 in range(1,self.D+1):
            cum=0
            for e1 in range(1,6*d1+1):
                cum += P(d1,e1) * self.EG(j,i+e1)
            ress.append(-cum)
        self.tab[i][j]=max(ress)  
        return self.tab[i][j]

In [162]:
stra_opt1 = strategie_optimal(10,100)

In [168]:
#print(stra_opt1.EG(95,95))
[stra_opt1.EG(i,99) for i in range(0,100)]
print(stra_opt1.EG(0,0))

0.06871904732303523


In [ ]:
#Q7

def strategie_aveugle():
    return 6


#def strategie_optimale(i,j):
    